In [77]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [78]:
data_paths = [
    {'csv':'champs.csv', 'competition': 'cup', 'country':''},
    {'csv':'dfbpokal.csv', 'competition': 'cup', 'country':'GER'},
    {'csv':'englandplayoffs.csv', 'competition': 'cup', 'country':'ENG'},
    {'csv':'facup.csv', 'competition': 'cup', 'country':'ENG'},
    {'csv':'leaguecup.csv', 'competition': 'cup', 'country':'ENG'},
    {'csv':'belgium.csv', 'competition': 'league', 'country':'BEL'},
    {'csv':'england.csv', 'competition': 'league', 'country':'ENG'},
    {'csv':'france.csv', 'competition': 'league', 'country':'FRA'},
    {'csv':'germany.csv', 'competition': 'league', 'country':'GER'},
    {'csv':'germany2.csv', 'competition': 'league', 'country':'GER'},
    {'csv':'greece.csv', 'competition': 'league', 'country':'GRE'},
    {'csv':'holland.csv', 'competition': 'league', 'country':'NED'},
    {'csv':'italy.csv', 'competition': 'league', 'country':'ITA'},
    {'csv':'mls.csv', 'competition': 'league', 'country':'USA'},
    {'csv':'portugal.csv', 'competition': 'league', 'country':'POR'},
    {'csv':'safrica.csv', 'competition': 'league', 'country':'SAF'},
    {'csv':'scotland.csv', 'competition': 'league', 'country':'SCO'},
    {'csv':'spain.csv', 'competition': 'league', 'country':'ESP'},
    {'csv':'turkey.csv', 'competition': 'league', 'country':'TUR'} 
]

desiredColumns = ['home', 'visitor', 'hgoal', 'vgoal', 'hcountry', 'vcountry', 'Date', 'competition']

combined_df = pd.DataFrame(columns=desiredColumns)

for x in data_paths:
    data_path = os.path.join('Data', 'engsoccerdata-master', 'engsoccerdata-master', 'data-raw', x['csv'])
    temp_df = pd.read_csv(data_path)
    temp_df['competition'] = x['competition']
    if x['country'] != '':
        temp_df['hcountry'] = x['country']
        temp_df['vcountry'] = x['country']
    
    # temp_df = temp_df[desiredColumns]
    
    combined_df = pd.concat([combined_df, temp_df[desiredColumns]], ignore_index=True)

combined_df.replace("NA", np.nan, inplace=True)
combined_df = combined_df.dropna()
    
print(combined_df.head)

C:\Users\mianw\AppData\Local\Temp\ipykernel_3292\1316158437.py:29: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(data_path)


<bound method NDFrame.head of                           home            visitor hgoal vgoal hcountry  \
0                  Sporting CP  Partizan Belgrade     3     3      POR   
1       Budapesti Voros Lobogo     RSC Anderlecht     6     3      HUN   
2              Servette Geneve        Real Madrid     0     2      SUI   
3              Rot-Weiss Essen       Hibernian FC     0     4      GER   
4               Djurgardens IF   Gwardia Warszawa     0     0      SWE   
...                        ...                ...   ...   ...      ...   
453030        Yeni Malatyaspor         Fenerbahce     0     5      TUR   
453031              Buyuksehyr        Trabzonspor     3     1      TUR   
453032           Karagumruk SK         Alanyaspor     0     1      TUR   
453033           Ad. Demirspor             Goztep     7     0      TUR   
453034               Hatayspor        Giresunspor     4     1      TUR   

       vcountry        Date competition  
0           SRB  1955-09-04         cup

In [79]:
def is_valid_date(date_str, date_format="%Y-%m-%d"):
    try:
        datetime.strptime(date_str, date_format)
        return True
    except ValueError:
        return False

In [80]:
combined_df['Date'] = combined_df['Date'].astype(str)

# Filter out invalid dates
combined_df['Valid_Date'] = combined_df['Date'].apply(is_valid_date)
invalid_dates = combined_df[~combined_df['Valid_Date']]
combined_df = combined_df[combined_df['Valid_Date']].drop(columns=['Valid_Date'])

# Split up the dates to use as separate columns
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

combined_df['year'] = combined_df['Date'].dt.year
combined_df['month'] = combined_df['Date'].dt.month
combined_df['day'] = combined_df['Date'].dt.day

combined_df = combined_df.drop(columns=['Date'])

print(combined_df.head)

<bound method NDFrame.head of                           home            visitor hgoal vgoal hcountry  \
0                  Sporting CP  Partizan Belgrade     3     3      POR   
1       Budapesti Voros Lobogo     RSC Anderlecht     6     3      HUN   
2              Servette Geneve        Real Madrid     0     2      SUI   
3              Rot-Weiss Essen       Hibernian FC     0     4      GER   
4               Djurgardens IF   Gwardia Warszawa     0     0      SWE   
...                        ...                ...   ...   ...      ...   
453030        Yeni Malatyaspor         Fenerbahce     0     5      TUR   
453031              Buyuksehyr        Trabzonspor     3     1      TUR   
453032           Karagumruk SK         Alanyaspor     0     1      TUR   
453033           Ad. Demirspor             Goztep     7     0      TUR   
453034               Hatayspor        Giresunspor     4     1      TUR   

       vcountry competition  year  month  day  
0           SRB         cup  1955

In [81]:
combined_df.to_csv('output.csv', index=False)